<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°03





**Objetivo**: Aplicar técnicas avanzadas de manipulación y análisis de datos con pandas sobre un conjunto real de datos de contenido de Netflix, reforzando buenas prácticas y métodos eficientes sin recurrir a `groupby`, `merge`, `pivot`, ni `join`.



**Dataset**:

Trabajaremos con el archivo `netflix_titles.csv`, que contiene información sobre los títulos disponibles en la plataforma Netflix hasta el año 2021.

| Variable       | Clase     | Descripción                                                                 |
|----------------|-----------|------------------------------------------------------------------------------|
| show_id        | caracter  | Identificador único del título en el catálogo de Netflix.                   |
| type           | caracter  | Tipo de contenido: 'Movie' o 'TV Show'.                                     |
| title          | caracter  | Título del contenido.                                                       |
| director       | caracter  | Nombre del director (puede ser nulo).                                       |
| cast           | caracter  | Lista de actores principales (puede ser nulo).                              |
| country        | caracter  | País o países donde se produjo el contenido.                                |
| date_added     | fecha     | Fecha en la que el título fue agregado al catálogo de Netflix.              |
| release_year   | entero    | Año de lanzamiento original del título.                                     |
| rating         | caracter  | Clasificación por edad (por ejemplo: 'PG-13', 'TV-MA').                      |
| duration       | caracter  | Duración del contenido (minutos o número de temporadas para series).        |
| listed_in      | caracter  | Categorías o géneros en los que está clasificado el contenido.              |
| description    | caracter  | Breve sinopsis del contenido.                                               |




In [ ]:
import pandas as pd

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...



### Parte 1: Limpieza y preparación

1. Revisar y describir el dataset:

   * ¿Cuántas filas y columnas tiene?
   * ¿Qué tipos de datos hay?
   * ¿Cuántos valores nulos hay por columna?

2. Transformar la columna `date_added` a tipo fecha.

3. Crear columnas auxiliares con `assign`:

   * Año (`year_added`)
   * Mes (`month_added`)



In [6]:
print("Dimensiones del dataset:", df.shape)
print("\nTipos de datos:")
print(df.dtypes)
print("\nValores nulos por columna:")
print(df.isna().sum())
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df = df.assign(
    year_added = df['date_added'].dt.year,
    month_added = df['date_added'].dt.month
)
df[['date_added', 'year_added', 'month_added']]

Dimensiones del dataset: (8807, 12)

Tipos de datos:
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

Valores nulos por columna:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


,date_added,year_added,month_added
0,2021-09-25,2021.0,9.0
1,2021-09-24,2021.0,9.0
2,2021-09-24,2021.0,9.0
3,2021-09-24,2021.0,9.0
4,2021-09-24,2021.0,9.0
5,2021-09-24,2021.0,9.0
6,2021-09-24,2021.0,9.0
7,2021-09-24,2021.0,9.0
8,2021-09-24,2021.0,9.0
9,2021-09-24,2021.0,9.0


## Parte 2: Técnicas avanzadas de pandas

4. Utilizar `.loc` para seleccionar películas (`type == 'Movie'`) que fueron agregadas después del año 2018.

5. Utilizar `str.contains()` y `str.extract()`:

   * Filtrar títulos que contienen la palabra 'love' (sin distinguir mayúsculas/minúsculas).
   * Extraer la duración en minutos para las películas desde la columna `duration`.

6. Aplicar `explode()` sobre la columna `listed_in` para obtener una fila por cada género.

7. Obtener un top 10 de géneros más frecuentes utilizando `value_counts()`.

8. Aplicar `where()` y `mask()` para marcar las películas de más de 120 minutos como contenido largo en una nueva columna.

9. Utilizar `.loc` para filtrar películas que cumplen con:

   * Más de 100 minutos de duración.
   * Rating igual a `'R'`.
   * País igual a `'United States'`.

10. Utilizar `.style` para formatear visualmente el top 10 de películas más largas.

In [24]:
peliculas_post_2018 = df.loc[(df['type'] == 'Movie') & (df['year_added'] > 2018)] #año 2018 > es 2019 >=, pero no se si eso pide
print("Películas agregadas después de 2018:", peliculas_post_2018.shape[0])

love_titulo = df[df['title'].str.contains("love", case=False, na=False)]
print("Títulos que contienen 'love':", love_titulo.shape[0])

df['duration_minutes'] = (
    df['duration']
    .str.extract(r'(\d+)')
    .astype(float))

duracion_peliculas = df[df['type'] == 'Movie'][['title','duration','duration_minutes']].head()

df_exploded = df.assign(listed_in=df['listed_in'].str.split(", ")).explode("listed_in")

top10_generos = df_exploded['listed_in'].value_counts().head(10)
print("\nTop 10 géneros más frecuentes:")
print(top10_generos)

df['long_content'] = df['duration_minutes'].where(df['duration_minutes'] <= 120, "Largo")
df['long_content'] = df['long_content'].mask(df['duration_minutes'] > 120, "Largo")

filtro_peliculas = df.loc[
    (df['type'] == 'Movie') &
    (df['duration_minutes'] > 100) &
    (df['rating'] == 'R') &
    (df['country'] == 'United States')
]
print("Películas filtradas (R, United States, >100min):", filtro_peliculas.shape[0])

top10_largas = (
    df[df['type'] == 'Movie']
    .sort_values(by='duration_minutes', ascending=False)
    [['title','duration_minutes','rating','country']]
    .head(10))
style_top10 = top10_largas.style.background_gradient(cmap="Blues").set_caption("Top 10 películas más largas de Netflix")
style_top10


Películas agregadas después de 2018: 3701
Títulos que contienen 'love': 196

Top 10 géneros más frecuentes:
listed_in
International Movies        2752
Dramas                      2427
Comedies                    1674
International TV Shows      1351
Documentaries                869
Action & Adventure           859
TV Dramas                    763
Independent Movies           756
Children & Family Movies     641
Romantic Movies              616
Name: count, dtype: int64
Películas filtradas (R, United States, >100min): 243


,title,duration_minutes,rating,country
4253,Black Mirror: Bandersnatch,312.000000,TV-MA,United States
717,Headspace: Unwind Your Mind,273.000000,TV-G,nan
2491,The School of Mischief,253.000000,TV-14,Egypt
2487,No Longer kids,237.000000,TV-14,Egypt
2484,Lock Your Girls In,233.000000,TV-PG,nan
2488,Raya and Sakina,230.000000,TV-14,nan
166,Once Upon a Time in America,229.000000,R,"Italy, United States"
7932,Sangam,228.000000,TV-14,India
1019,Lagaan,224.000000,PG,"India, United Kingdom"
4573,Jodhaa Akbar,214.000000,TV-14,India




### Pregunta Desafío

11. ¿Cuáles son las combinaciones más frecuentes de género y rating en el dataset?
    (Sugerencia: utilizar `value_counts` con `subset=["genre", "rating"]` después de aplicar `explode()`).



### Bonus: Análisis de duplicados y limpieza

12. ¿Existen películas con el mismo nombre (`title`) pero con distinto año de lanzamiento (`release_year`)?
13. ¿Cuántos títulos únicos hay en total en la columna `title`?





In [23]:
combinaciones_genero_rating = (
    df_exploded.value_counts(subset=["listed_in", "rating"])
    .reset_index(name="Frecuencia")
    .sort_values("Frecuencia", ascending=False)
)

print(combinaciones_genero_rating.head(10)) #t10

titulos_unicos = df['title'].nunique()
print("Cantidad de títulos únicos:", titulos_unicos)

#Por lo tanto no hay duplicados, pues 8807 es el tanaño del dataset

                listed_in rating  Frecuencia
0    International Movies  TV-MA        1130
1    International Movies  TV-14        1065
2                  Dramas  TV-MA         830
3  International TV Shows  TV-MA         714
4                  Dramas  TV-14         693
5  International TV Shows  TV-14         472
6                Comedies  TV-14         465
7               TV Dramas  TV-MA         434
8                Comedies  TV-MA         431
9                  Dramas      R         375
Cantidad de títulos únicos: 8807
